In [1]:
!pip install skorch

     |████████████████████████████████| 133kB 15.5MB/s 


In [2]:
import torch.nn as nn
from skorch import NeuralNetClassifier
import torch
import torch.optim as optim
from transformers import *
import pickle
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [43]:
with open("x_train.pkl", "rb") as file:
    x_train = pickle.load(file)

with open("x_valid.pkl", "rb") as file:
    x_valid = pickle.load(file)

with open("x_test.pkl", "rb") as file:
    x_test = pickle.load(file)

with open("y_train.pkl", "rb") as file:
    y_train = pickle.load(file)

with open("y_valid.pkl", "rb") as file:
    y_valid = pickle.load(file)

with open("y_test.pkl", "rb") as file:
    y_test = pickle.load(file)

In [68]:
class ConvNet(nn.Module):
    def __init__(self):
        print('net init')
        super(ConvNet, self).__init__()

        self.conv = nn.Sequential(
            # 360 x 1
            nn.Conv1d(1, 32, kernel_size=3, stride=3, padding=0),
            nn.BatchNorm1d(32),
            nn.ReLU(inplace=True),
            nn.Conv1d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),

            # 120 x 32
            nn.Conv1d(32, 48, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(48),
            nn.ReLU(),
            nn.Conv1d(48, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU())

        self.fully_connected_layer = nn.Sequential(
            nn.Dropout(p=0.8),
            nn.Linear(10944, 720),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),
            nn.Linear(720, 2)
        )

    def forward(self, x):
        
        x = x.view(x.shape[0], 1, -1)
        
        x = self.conv(x)

        x = x.view(x.size(0), -1)
        x = self.fully_connected_layer(x)

        return x

In [69]:
net = NeuralNetClassifier(
        ConvNet,
        max_epochs=14,
        lr=0.0001,
        batch_size=1024,
        verbose=4,
        criterion=nn.CrossEntropyLoss,
        optimizer=optim.Adam,
        device=device)

In [70]:
pipe = Pipeline(steps=[
    ('RoleLineMerge', RoleLineMergeTransformer()),
    ('DropUselessColumns', DropUselessColumnsTransformer(extra_input=[ExtraInputType.FIRST_BLOOD, ExtraInputType.FIRST_DRAGON, ExtraInputType.FIRST_HERALD, ExtraInputType.FIRST_TOWER])),
    ('MakeVector', MakeVectorTransformer(top_n=50, augmented=False)),
    ('Model', net)
])

init called
init called
init called


In [73]:
pipe.fit(x_train, y_train['Win'].astype('int64'))

Re-initializing module.
net init
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6835       0.4944        0.6927  2.8493
      2        0.6534       0.6486        0.6536  2.7225
      3        0.6439       0.6639        0.6312  2.7182
      4        0.6353       0.6662        0.6251  2.7244
      5        0.6309       0.6670        0.6228  2.7385
      6        0.6275       0.6647        0.6230  2.7219
      7        0.6261       0.6670        0.6223  2.7934
      8        0.6252       0.6687        0.6221  2.7151
      9        0.6178       0.6697        0.6194  2.7478
     10        0.6203       0.6702        0.6226  2.7297
     11        0.6153       0.6701        0.6198  2.7145
     12        0.6164       0.6710        0.6239  2.7665
     13        0.6125       0.6699        0.6216  2.7301
     14        0.6122       0.6670        0.6242  2.7263


Pipeline(memory=None,
         steps=[('RoleLineMerge', RoleLineMergeTransformer()),
                ('DropUselessColumns',
                 DropUselessColumnsTransformer(extra_input=[<ExtraInputType.FIRST_BLOOD: 1>,
                                                            <ExtraInputType.FIRST_DRAGON: 2>,
                                                            <ExtraInputType.FIRST_HERALD: 4>,
                                                            <ExtraInputType.FIRST_TOWER: 3>])),
                ('MakeVector',
                 MakeVectorTransformer(augmented=False, top_n=50)),
                ('Model',
                 <class 'skor...
      (7): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
      (9): Conv1d(48, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (10): ReLU()
    )
    (fully_connected_layer): Sequential(
      (0): Dropout(p=0.8, inplace=False)
      (1): Linear(in_features=10944, out_features=720,

In [74]:
predictions = pipe.predict(x_test)
print(accuracy_score(y_true=y_test['Win'].astype(float), y_pred=predictions))

0.6724191063174114
